# Layer Normalization

Layer Normalization은 딥러닝 모델에서 각 층의 출력을 정규화하는 기법으로, 훈련 안정성과 수렴 속도를 개선하는 데 도움을 주는 기법이다.

단어 그대로 각 층을 Normalize한다고 보면 된다.

Layer Normalization에서 중요한 것은 `어디`와 `어떻게`이다.

먼저 `어떻게`에 대해 알아보겠다.<br>
정규화 수식은 다음과 같다.

$$
\text{LayerNorm}(x) = \frac{x - \mu}{\sigma} \cdot \gamma + \beta
$$

정규화 대상인 입력 `x`에 대하여 평균과 표준편차를 사용하여 정규화를 수행한다.<br>
정규화가 된 경우 평균이 0, 표준편차가 1이 되는 정규분포의 형태를 띄게 된다.<br>

이렇게 정규화를 수행한 뒤, $\gamma$와 $\beta$로 affine transform을 수행한다.<br>
이에 대해 생각해보면 먼저 정규분포의 형태로 데이터를 정규화한 뒤, 아핀 변환으로 분포를 데이터와 태스크에 적합하도록 조정하는 것이다.<br>
단순히 정규화만 수행하는게 아니라는 것이 중요하다.

코드로 `nn.LayerNorm`을 확인해보면 weight와 bias가 나오는데, 각각 $\gamma$와 $\beta$를 의미한다.<br>
`nn.LayerNorm` 객체를 불러오고 약간의 조작을 통해 연산을 확인해보겠다.

In [18]:
import torch
import torch.nn as nn

x = torch.tensor([0.2, 0.2, 0.2, 0.2, 0.2])

ln = nn.LayerNorm(5)
# ln.weight.data = torch.tensor([1., 2., 3., 4., 5.], requires_grad=True)

# 직관적인 확인을 위해 beta 값 만을 변화시켜 확인하겠다.
ln.bias.data = torch.tensor([2.,4.,6.,8.,10.], requires_grad=True)

ln(x)

tensor([ 2.,  4.,  6.,  8., 10.], grad_fn=<NativeLayerNormBackward0>)

이제 `어디`에 대해 알아보겠다.<br>
각 층을 대상으로 정규화라고 하였는데, '층'이라는 표현은 다소 모호하다. <br>

LayerNorm은 보통 RNN, transformer 모델에서 적용한다.<br>
따라서 입력 텐서의 크기가 `(Batch, Sequence_length, hidden_dim)`라고 가정하겠다.<br>
이때 각 단어별 벡터를 대상으로 정규화를 수행한다.<br>
따라서 `nn.LayerNorm`은 `hidden_dim`의 크기를 할당하여 객체를 불러온다.

In [36]:
import torch
import torch.nn as nn

# 배치 크기, 시퀀스 길이, 히든 차원
batch, seq_len, h_dim = 3, 12, 768
input_tensor = torch.rand(batch, seq_len, h_dim)

layer_norm = nn.LayerNorm(normalized_shape=h_dim)

ln_tensor = layer_norm(input_tensor)
print(sum(ln_tensor[0,0,:]))

tensor(5.9128e-05, grad_fn=<AddBackward0>)


이때 `normalized_shape`에 오는 크기는 입력 차원의 뒤에서부터 순서대로 와야한다.<br>
LayerNorm의 대상을 정하는 것이므로 이미지 텐서를 예시로 들면 (열, 행, 차원, 배치) 순서로 정규화 차원이 확장되는 것이다.<br>
코드로 자세히 확인하겠다.

Simple Example

In [46]:
import torch
import torch.nn as nn

N, C, H, W = 2, 3, 4, 4  
input_tensor = torch.rand(N, C, H, W)  

# Layer Normalization을 위한 nn.LayerNorm 모듈
layer_norm_1 = nn.LayerNorm(normalized_shape=W)
layer_norm_2 = nn.LayerNorm(normalized_shape=(H, W))
layer_norm_3 = nn.LayerNorm(normalized_shape=(C, H, W))
layer_norm_4 = nn.LayerNorm(normalized_shape=(N, C, H, W))

# 각 Layer Normalization을 적용 수행
output_tensor = layer_norm_1(input_tensor)
output_tensor = layer_norm_2(input_tensor)
output_tensor = layer_norm_3(input_tensor)
output_tensor = layer_norm_4(input_tensor)

# 오류 없이 수행되는 것을 확인할 수 있다.

In [53]:
print(layer_norm_3.weight.data.shape,
layer_norm_3.bias.data.shape)

print(layer_norm_4.weight.data.shape,
layer_norm_4.bias.data.shape)

torch.Size([3, 4, 4]) torch.Size([3, 4, 4])
torch.Size([2, 3, 4, 4]) torch.Size([2, 3, 4, 4])


# 수식 구현

In [42]:
ln = nn.LayerNorm(5)
input = torch.randn(1, 5)

output = ln(input)
print(output)
print(ln.weight)
print(ln.bias)

tensor([[ 0.0793, -1.4627, -0.0971, -0.1949,  1.6753]],
       grad_fn=<NativeLayerNormBackward0>)
Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


In [44]:
mean = input.mean(-1, keepdim=True)
var = (input - mean).pow(2).mean(-1, keepdim=True)
norm = (input - mean) / torch.sqrt(var + ln.eps) * ln.weight + ln.bias

print(norm)

tensor([[ 0.0793, -1.4627, -0.0971, -0.1949,  1.6753]], grad_fn=<AddBackward0>)


In [45]:
# nn.LayerNorm과의 비교
torch.allclose(output, norm)

True